In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
#Leemos los CSV
results = pd.read_csv("./data/survey_results_public.csv",index_col="ResponseId")
results_schema = pd.read_csv("./data/survey_results_schema.csv")

In [ ]:
results.head(5)

In [ ]:
languages = "LanguageHaveWorkedWith"
salary = "ConvertedCompYearly"
#Renombramos las columnas que nos interesan
results.rename(columns={languages: "Languages",salary:"Salary"},inplace=True)
#Eliminamos las filas que tengan NaN
results.dropna(subset=["Salary","Languages"],inplace=True)
#Limitamos DataFrame a las columns que nos interesan y ordenamos por el salario
results = results.loc[:,["Country","Salary","Languages"]].sort_values(by="Salary")
#Realizamos un filtro para el salario
filtro = (results["Salary"] >= 10000) & (results["Salary"] <= 3e6)
#Aplicamos el filtro a los datos
results = results[filtro]


In [ ]:
lg = results["Languages"]
#Inicializamos una lista con algunos lenguajes
all_languages = lg[80568].split(";")
#Por cada fila en los lenguajes
for row in lg:
    #Y por cada lenguaje en cada fila
    for lang in row.split(";"):
        #Lo añadimos a la lista
        all_languages.append(lang)
#Hacemos un set para tener solamente los lenguajes que sean diferentes
#Convertimos el set en una lista
all_languages = list(set(all_languages))
#Ordenamos alfabéticamente
all_languages.sort()
#Creamos un DataFrame, para los datos creamos un array de ceros del tamaño de la lista de lenguajes
#El index son los lenguajes
#Y la columna se nombra "Salary"
mean_salary = pd.DataFrame(data=np.zeros(len(all_languages)),index=all_languages,columns=["Salary"])


In [ ]:
import re
#Para cada lenguaje en la lista de lenguajes
for lang in all_languages:
    try:
        #Realizamos una máscara, y obtenemos las filas que contienen ese lenguaje
        mask = results["Languages"].str.contains(re.escape(lang))
        #Hacemos un promedio de los salarios en las filas obtenidas en mask
        average = results.loc[mask,"Salary"].mean()
        #Colocamos el promedio en el "Salary" para la fila del lenguaje adecuado
        mean_salary.loc[lang,"Salary"] = average
    except Exception:
        print(lang)
"""
ocupamos el módulo de expresiones regulares
para evitar que haya errores al momento
de evaluar cada lenguaje en las filas,
por ello ocupamos re.escape, debido a que
C++ y posiblemente algún otro lenguaje cause un error
"""

In [ ]:
colors = [
    "#03071E",
    "#370617",
    "#6A040F",
    "#9D0208",
    "#D00000",
    "#DC2F02",
    "#E85D04",
    "#F48C06",
    "#FAA307",
    "#FFBA08"
]

fontdict = {
    'family': 'serif',
    'color': 'darkred',
    'weight': 'normal',
    'size': 16
}



In [16]:
#hacemos que nos muestre el gráfico, no incrustado en la libreta, si no en una ventana adicional
%matplotlib qt5

#aplicamos un estilo, para ver los estilos disponibles, usamos plt.style.avaible()
plt.style.use("seaborn-v0_8-darkgrid")
#ordenamos el DF por el salario

mean_salary.sort_values(by="Salary",inplace=True)
#hacemos un gráfico de barras horizontal, le pasamos los lenguajes como index, los datos que están en "Salary"
plt.barh(mean_salary.index,mean_salary["Salary"],color=colors)
#Agregamos una barra vertical para representar el promedio de los salarios promedio, que va desde 0 hasta la longitud de "Salary"
#Aplicamos color rojo, linea punteada y agregamos un label
plt.vlines(mean_salary["Salary"].mean(),0,len(mean_salary["Salary"]),color="red",linestyles="dashed",label="Mean Salary")

#Agregamos el titulo del gráfico
plt.title("Programming Languages Yearly Compentation in 2023")
#Ponemos la etiqueta para los ejes X y Y
plt.xlabel("Mean Salary in $")
plt.ylabel("Programming Languages")

#Agregamos el símbolo de $ a los datos del eje X
plt.gca().xaxis.set_major_formatter('${x:1.0f}')
#Mostramos la leyenda
plt.legend()
#Espacía los diferentes elementos para que no se sobrepongan
plt.tight_layout()
#Mostramos el gráfico
plt.show()
